In [19]:
import numpy as np
import cv2 as cv

In [20]:
UBIT = 'pravi'
np.random.seed(sum([ord(a) for a in UBIT]))

In [21]:
def extract_sift(filepath,output_file):
    img = cv.imread(filepath)
    gray = cv.imread(filepath,0)
    sift = cv.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    kpimg = cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv.imwrite(output_file,kpimg)

In [22]:
extract_sift('tsucuba_left.png','task2_sift1.jpg')

In [23]:
extract_sift('tsucuba_right.png','task2_sift2.jpg')

In [24]:
img1 = cv.imread('tsucuba_left.png')
img2 = cv.imread('tsucuba_right.png')
img1_gray = cv.imread('tsucuba_left.png',0)
img2_gray = cv.imread('tsucuba_right.png',0)

sift = cv.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1_gray,None)
kp2, des2 = sift.detectAndCompute(img2_gray,None)

# BFMatcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
pts1 = []
pts2 = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,flags=2)
cv.imwrite('task2_matches_knn.jpg',img3)

True

In [25]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)

random_pt1 = []
random_pt2 = []
for i in [np.random.randint(0,len(good)-1) for x in range(10)]:
    random_pt1.append(pts1[i])
    random_pt2.append(pts2[i]) 

ran_pt1 = np.int32(random_pt1)
ran_pt2 = np.int32(random_pt2)


F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

print(F)

[[ 3.88435667e-06 -2.28004165e-04  4.41240231e-02]
 [ 2.41983065e-04 -1.90250881e-05  9.94644488e-02]
 [-4.57546817e-02 -1.02452413e-01  1.00000000e+00]]


In [26]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape[:2]
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

In [27]:
# Find epilines corresponding to points in right image (second image) and drawing its lines on left image
lines1 = cv.computeCorrespondEpilines(ran_pt2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,ran_pt1,ran_pt2)

# Find epilines corresponding to points in left image (first image) and drawing its lines on right image
lines2 = cv.computeCorrespondEpilines(ran_pt1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,ran_pt2,ran_pt1)

In [28]:
cv.imwrite('task2_epi_right.jpg',img3)

True

In [33]:
cv.imwrite('task2_epi_img4.jpg',img4)
cv.imwrite('task2_epi_left.jpg',img5)
cv.imwrite('task2_epi_img6.jpg',img6)

True

In [34]:
stereo = cv.StereoBM_create(numDisparities=80, blockSize=15)
disparity = stereo.compute(img1_gray,img2_gray)
cv.imwrite('task2_disparity.jpg',disparity)

True

In [35]:
stereo = cv.StereoSGBM_create(minDisparity = 16,
        numDisparities = 80,
        uniquenessRatio = 10,
        speckleWindowSize = 100,
        speckleRange = 32,
        disp12MaxDiff = 1,
        P1 = 8*3*3**2,
        P2 = 32*3*3**2
    )


In [36]:
disparity = stereo.compute(img1_gray,img2_gray)/16
cv.imwrite('task2_disparity.jpg',disparity)

True